In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
import pyrtools as pt
import plenoptic as po
import scipy.io as sio
import seaborn as sns
import h5py

torch.__version__
numpy = lambda x: x.detach().cpu().numpy().squeeze()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def relTol(a,b):
    return(abs(a-b)/(abs(a)+abs(b)))

def absTol(a,b):
    return(abs(a-b))

def reportTol(a,b):
    return (max(relTol(a,b)),max(absTol(a,b)))


In [2]:
f = h5py.File('/Users/kathrynbonnen/Documents/MATLAB/textureSynth/pyramid.mat','r')

Nsc = np.int64(f['Nsc'][()][0][0])
Nor = np.int64(f['Nor'][()][0][0])
im0 = torch.Tensor(f['im0'][()]).T.unsqueeze(0).unsqueeze(0);
pind_m = f['pind0'][()]
pyr0 = f['pyr0'][()].flatten()


f = h5py.File('/Users/kathrynbonnen/Documents/MATLAB/textureSynth/parents.mat','r')
rparents_mat = f['rparents'][()].T
cousins_mat = f['cousins'][()].T
print(rparents_mat[0])

[ -63.42752039  116.7691824  -929.83217244  465.4683692  -260.5042593 ]


In [3]:
matlab_pyr = po.simulate.Steerable_Pyramid_Freq.vector_to_pyramid(pyr0, im0.shape, Nsc, Nor)
model = po.simulate.Portilla_Simoncelli(im0.shape[-2:],n_scales=4,n_orientations=4,Na=3,inputImageIsPyramid=True)
y = model.forward((matlab_pyr,im0))


True
statg0:  torch.Size([6, 1]) 6
magMeans0:  torch.Size([18, 1]) 18
ace:  torch.Size([3, 3, 4, 4]) 144
skew0p:  torch.Size([5, 1]) 5
kurt0p:  torch.Size([5, 1]) 5
acr:  torch.Size([3, 3, 5]) 45
C0:  torch.Size([4, 4, 5]) 80
Cx0:  torch.Size([4, 4, 4]) 64
Cr0:  torch.Size([8, 8, 5]) 320
Crx0:  torch.Size([8, 8, 4]) 256
vHPR0:  tensor(364.8426) 1


In [14]:
model_py = po.simulate.Portilla_Simoncelli(im0.shape[-2:],n_scales=4,n_orientations=4,Na=3)
y_py = model_py.forward(im0)

(relTol(y,y_py)>1e-5).nonzero()


False
statg0:  torch.Size([6, 1]) 6
magMeans0:  torch.Size([18, 1]) 18
ace:  torch.Size([3, 3, 4, 4]) 144
skew0p:  torch.Size([5, 1]) 5
kurt0p:  torch.Size([5, 1]) 5
acr:  torch.Size([3, 3, 5]) 45
C0:  torch.Size([4, 4, 5]) 80
Cx0:  torch.Size([4, 4, 4]) 64
Cr0:  torch.Size([8, 8, 5]) 320
Crx0:  torch.Size([8, 8, 4]) 256
vHPR0:  tensor(364.8424) 1


tensor([[690],
        [722],
        [754],
        [786]])

In [34]:
tmp = sio.loadmat('../data/plenoptic-test-files/PortillaSimoncelliMatlab0.mat')

In [35]:
matfile = sio.loadmat('../data/plenoptic-test-files/PortillaSimoncelliMatlab'+str(0))
im0 = torch.Tensor(matfile['im0']).unsqueeze(0).unsqueeze(0)
paramsMat = np.array(matfile['paramsVector']).flatten()
model = po.simulate.Portilla_Simoncelli(im0.shape[-2:],n_scales=4,n_orientations=4,Na=9)
paramsPython = model.forward(im0)
paramsPython = po.to_numpy(paramsPython)


torch.Size([32, 32, 2])
tensor([[   22.4766,   339.4022,   457.5108,  ...,  -339.4024,  -457.5112,
          -307.6158],
        [  339.4023,  5125.1748,  6908.6890,  ..., -5125.1748, -6908.6895,
         -4645.1875],
        [  457.5107,  6908.6895,  9312.8516,  ..., -6908.6895, -9312.8516,
         -6261.6714],
        ...,
        [ -339.4023, -5125.1748, -6908.6895,  ...,  5125.1748,  6908.6890,
          4645.1875],
        [ -457.5108, -6908.6895, -9312.8516,  ...,  6908.6895,  9312.8516,
          6261.6714],
        [ -307.6156, -4645.1875, -6261.6714,  ...,  4645.1875,  6261.6709,
          4210.1533]])
statg0:  torch.Size([6, 1]) 6
magMeans0:  torch.Size([18, 1]) 18
ace:  torch.Size([9, 9, 4, 4]) 1296
skew0p:  torch.Size([5, 1]) 5
kurt0p:  torch.Size([5, 1]) 5
acr:  torch.Size([9, 9, 5]) 405
C0:  torch.Size([4, 4, 5]) 80
Cx0:  torch.Size([4, 4, 4]) 64
Cr0:  torch.Size([8, 8, 5]) 320
Crx0:  torch.Size([8, 8, 4]) 256
vHPR0:  tensor(74.0163) 1
1.4901161193847656e-08
32.0
(array(